In [ ]:
!pip install torchcodec
!pip install soundfile
!pip install scipy
!pip install gTTS
!pip install pydub

In [ ]:
from diffwave.inference import predict as diffwave_predict
import torch
from scipy.io.wavfile import write
import numpy as np
from data_generation import generate_audio_waveforms

Perform the following to get the mel spectrograms

In [ ]:
sentences = [
    "The patient is speaking softly about the weather.",
    "Please bring the red cup from the kitchen."
]

output_dir = "./audio_wav"

generate_audio_waveforms(sentences, output_dir)

In [37]:
!python -m diffwave.preprocess ./audio_wav

Preprocessing: 100%|██████████████████████████████| 3/3 [00:02<00:00,  1.44it/s]


In [38]:
mel = np.load("./audio_wav/sentence_2.wav.spec.npy")  # shape: (n_mels, time)

# # Convert to PyTorch tensor
mel = torch.from_numpy(mel).float()


model_dir = './pretrained_model/diffwave-ljspeech-22kHz-1000578.pt'
audio, sample_rate, latent_vars = diffwave_predict(mel, model_dir, fast_sampling=True, expose_latent_vars=True, compute_jacobian=False)
write("output.wav", sample_rate, audio.squeeze().cpu().numpy())

Try injecting latent vars into inference with correct mel spectrogram

In [34]:
audio, sample_rate, __ = diffwave_predict(mel, model_dir, fast_sampling=True, inject_latent_var=(3, latent_vars[3]))
write("injected_output.wav", sample_rate, audio.squeeze().cpu().numpy())

## Steps to train to fit brain signals into diffusion pipeline
1. Get mel spectrograms of all of the GT sentences
2. Use those to get latent variables to train
3. Train model(s) to give use mel spectrograms and latent variables

### Notes on setup
- Must choose whether we use fast sampling or not